点多边形测试

In [ ]:
import cv2 as cv
import numpy as np


src = cv.imread(PictureAddress)                                                                # 读取图像文件
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                                                    # 创建可自动调整大小的窗口
cv.imshow("input", src)                                                                        # 显示原始图像
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)                                                     # 转换为灰度图
ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)                    # OTSU二值化
cv.imshow("binary", binary)                                                                    # 显示二值图像

# 轮廓发现
image = np.zeros(src.shape, dtype=np.float32)                                                  # src.shape 继承原始图像的宽高和通道数，np.float32 确保支持负距离值存储
contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)        # RETR_EXTERNAL 仅检测最外层轮廓
h, w = src.shape[:2]                                                                           # 获取图像的宽高
for row in range(h):
    for col in range(w):
        # cv.pointPolygonTest()用于计算点与多边形关系，返回三种值：正数：点到轮廓内部的最近距离，负数：点到轮廓外部的最近距离，零：点在轮廓上
        dist = cv.pointPolygonTest(contours[0], (col, row), True)                # (col, row)：待测试点的坐标（注意是(x,y)格式）
        if dist == 0:
            image[row, col] = (255, 255, 255)                                                  # 边界点（dist=0）：纯白色
        if dist > 0:
            image[row, col] = (255 - dist, 0, 0)                                               # 蓝色渐变
        if dist < 0:
            image[row, col] = (0, 0, 255 + dist)                                               # 红色渐变

dst = cv.convertScaleAbs(image)                                                                # cv.convertScaleAbs()：执行线性变换并取绝对值,自动将结果缩放到0-255范围
dst = np.uint8(dst)                                                                            # 强制转换为8位无符号整型,确保数据格式符合图像显示要求

# 显示
cv.imshow("contours_analysis", dst)
#cv.imwrite("D:/contours_analysis.png", dst)

cv.waitKey(0)                                                                                  # 等待按键
cv.destroyAllWindows()                                                                         # 关闭所有窗口

凸包检测

In [ ]:
import cv2 as cv
import numpy as np

src = cv.imread(PictureAddress)                                                                # 读取图像文件
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                                                    # 创建可自动调整大小的窗口
cv.imshow("input", src)                                                                        # 显示图像
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)                                                     # 转换为灰度图
ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)                    # OTSU二值化
k = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))                                            # 创建3x3矩形核
binary = cv.morphologyEx(binary, cv.MORPH_OPEN, k)                                             # 开运算（先腐蚀后膨胀）
# cv.imshow("binary", binary)

# 轮廓发现
contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)        # 从二值图像中提取所有外部轮廓
for c in range(len(contours)):                                                                 # 遍历所有检测到的轮廓
    # cv.isContourConvex()：基于Sklansky算法判断轮廓凸性
    ret = cv.isContourConvex(contours[c])                                                      # 检查轮廓是否为凸包，返回值：布尔值（True表示凸轮廓）
    # cv.convexHull()：基于Andrew's monotone chain算法
    points = cv.convexHull(contours[c])                                                        # Andrew's单调链算法计算轮廓的凸包
    total = len(points)
    # 获取凸包相邻顶点坐标（实现多边形边遍历）   [i]选择第i个顶点  [0]获取去掉单维后的坐标值
    for i in range(len(points)):
        x1, y1 = points[i%total][0]                                                            # points是形状为(N,1,2)的NumPy数组，存储轮廓顶点坐标，通过[0]操作提取去掉单维后的二维坐标
        x2, y2 = points[(i+1)%total][0]                                                        # 通过取模运算%total实现环形访问（首尾顶点自动连接），例：当i=last_index时，(i+1)%total=0
        cv.circle(src, (x1, y1), 4, (255, 0, 0), 2, 8, 0)                                      # 绘制蓝色顶点，顶点半径4px确保可视化清晰度
        cv.line(src, (x1, y1), (x2, y2), (0, 0, 255), 2, 8, 0)                                 # 绘制红色边线
    print(points)                                                                              # 输出凸包顶点坐标
    print("convex : ", ret)                                                                    # 输出凸性检测结果

# 显示
cv.imshow("contours_analysis", src)
#cv.imwrite("D:/contours_analysis.png", src)

cv.waitKey(0)                                                                                  # 等待按键
cv.destroyAllWindows()                                                                         # 关闭所有窗口

Hu矩轮廓匹配

In [ ]:
import cv2 as cv
import numpy as np


# 原始图像 → 灰度化 → 二值化 → 轮廓提取 → 计算Hu矩 → 形状匹配 → 可视化结果
#                      ↑
#               (Otsu自动阈值)


src = cv.imread(Picture1Address)                                                              # 读取图像文件1
cv.imshow("input1", src)                                                                      # 显示图像1
src2 = cv.imread(Picture2Address)                                                             # 读取图像文件2
cv.imshow("input2", src2)                                                                     # 显示图像2

# 提取图像轮廓
def contours_info(image): 
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)                                              # 转为灰度图
    ret, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)               # Otsu算法自适应确定二值化阈值
    contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)   # 轮廓发现，RETR_EXTERNAL只检测最外层轮廓
    return contours
# 轮廓提取
contours1 = contours_info(src)                                                                # 提取图像1轮廓
contours2 = contours_info(src2)                                                               # 提取图像2轮廓
# 几何矩计算与hu矩计算 
mm2 = cv.moments(contours2[0])                                                                # 通过cv.moments()计算src2的几何矩（24个特征）
hum2 = cv.HuMoments(mm2)                                                                      # 转换为Hu矩（7个旋转/缩放不变特征）
# 轮廓匹配
for c in range(len(contours1)):                                                               # 遍历contours1
    mm = cv.moments(contours1[c])                                                             # 通过cv.moments()计算src1的几何矩
    hum = cv.HuMoments(mm)                                                                    # 转换为Hu矩
    # 形状匹配
    dist = cv.matchShapes(hum, hum2, cv.CONTOURS_MATCH_I1, 0)                                 # 计算形状距离，CONTOURS_MATCH_I1：使用第一种相似度度量方法
    if dist < 1:                                                                              # 距离值dist越小表示形状越相似，这里选1.0是经验值，可根据实际调整
        cv.drawContours(src, contours1, c, (0, 0, 255), 2, 8)
    print("dist %f"%(dist))

# 显示
cv.imshow("contours_analysis", src)
#cv.imwrite("D:/contours_analysis.png", src)

cv.waitKey(0)                                                                                 # 等待按键
cv.destroyAllWindows()                                                                        # 关闭所有窗口

最大内接圆

In [ ]:
from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np

# Create an image (创建400x400像素的纯黑图像（4*r=400）)
r = 100
src = np.zeros((4*r, 4*r), dtype=np.uint8)
# Create a sequence of points to make a contour（创建一系列点以形成轮廓）
#定义了一个六边形的顶点坐标，常用于计算机图形学中的多边形绘制
vert = [None]*6                                                           # 使用[None]创建单元素列表，通过*6运算符复制6次
# 六边形顶点坐标的数学表达式六边形顶点坐标的数学表达式 
vert[0] = (3*r//2, int(1.34*r))     # 第一象限顶点                         # 3*r//2：计算x坐标（半径的1.5倍取整），int(1.34*r)：计算y坐标（1.34倍半径取整)，1.34≈2-sqrt(3)/2（六边形的垂直偏移系数）
vert[1] = (1*r, 2*r)                # 第二象限上部顶点                      
vert[2] = (3*r//2, int(2.866*r))    # 第二象限下部顶点                      # 垂直间距：约0.866*r（sin(60°)*r）
vert[3] = (5*r//2, int(2.866*r))    # 第三象限顶点 
vert[4] = (3*r, 2*r)                # 第四象限顶点
vert[5] = (5*r//2, int(1.34*r))     # 第一象限下部顶点

# 补充
# OpenCV绘制六边形示例
#pts = np.array(vert, np.int32)
#cv.polylines(src, [pts], True, (0,255,0), thickness=2)

# Draw it in src（在原图中绘制它）
for i in range(6):                                                         # range(6) 遍历六边形的6个顶点，
    cv.line(src, vert[i],  vert[(i+1)%6], ( 255 ), 3)                      # vert[i]：当前顶点坐标，vert[(i+1)%6]：通过取模运算实现环形索引（第6点连接回第1点），(i+1)%n是处理闭合多边形的经典模式

# Get the contours（获取轮廓）
contours, _ = cv.findContours(src, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)   # cv.RETR_TREE：检索所有轮廓并重建层级关系

# Calculate the distances to the contour
raw_dist = np.empty(src.shape, dtype=np.float32)                           # 使用NumPy创建了一个与输入图像src维度相同的空浮点数组，是图像处理中常见的预分配内存操作
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        # cv.pointPolygonTest() 执行点与多边形的几何关系检测，返回三种值：正数：点到轮廓内部的最近距离，负数：点到轮廓外部的最近距离，零：点在轮廓上
        # (j,i)：测试点坐标（注意OpenCV的(x,y)对应numpy的[行,列]即(i,j); contours[0]：输入的多边形轮廓（通常来自findContours结果）
        raw_dist[i,j] = cv.pointPolygonTest(contours[0], (j,i), True)      # True：启用带符号距离计算（正值为内部，负值为外部）

# 获取最大值即内接圆半径，中心点坐标
# minVal：矩阵中的最小值（最近距离） maxVal：矩阵中的最大值（最远距离）  _：忽略的最小值位置（用占位符_表示）  maxDistPt：最大值坐标点（Point类型）
minVal, maxVal, _, maxDistPt = cv.minMaxLoc(raw_dist)
# 取绝对值，确保距离值为正
minVal = abs(minVal)
maxVal = abs(maxVal)                                                       # 在形状分析中，maxVal常表示物体的最大内接圆半径

# Depicting the  distances graphically（用图形描绘距离）
drawing = np.zeros((src.shape[0], src.shape[1], 3), dtype=np.uint8)        # 创建了一个用于图像绘制的空白RGB画布; src.shape[0]：图像高度（行数);src.shape[1]：图像宽度（列数）
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
# 根据距离值(raw_dist)的正负进行不同通道的着色; 采用反相处理(255-x)增强视觉对比度
        if raw_dist[i,j] < 0:                                              # 负距离值：蓝色通道(B)渐变（距离越近值越大）,使用minVal进行归一化到[0,255]
            drawing[i,j,0] = 255 - abs(raw_dist[i,j]) * 255 / minVal
        elif raw_dist[i,j] > 0:                                            # 正距离值：红色通道(R)渐变（距离越近值越大）,使用maxVal进行归一化到[0,255]
            drawing[i,j,2] = 255 - raw_dist[i,j] * 255 / maxVal
        else:                                                              # 零值点显示为白色(RGB=255,255,255)
            drawing[i,j,0] = 255
            drawing[i,j,1] = 255
            drawing[i,j,2] = 255

# max inner circle
# maxDistPt：圆心坐标(Point类型);int(maxVal)：圆半径（取整后的最大距离值）;0：位移参数（默认0）
cv.circle(drawing,maxDistPt, int(maxVal),(255,255,255), 1, cv.LINE_8, 0)   # 半径使用maxVal保证包含所有有效区域
#cv.imshow('Source', src)
cv.imshow('Distance and inscribed circle', drawing)

cv.waitKey(0)                                                              # 等待按键
cv.destroyAllWindows()                                                     # 关闭所有窗口

几何矩计算轮廓中心与横纵比过滤

In [ ]:
import cv2 as cv
import numpy as np


# 二值图像 → 轮廓提取 → 几何分析 → 特征计算 → 分类标注
#               │              ├→ 宽高比 → 形状分类
#               │               └→ 质心坐标 → 精确定位
#               └→ 旋转矩形 → 顶点坐标 → 可视化

# 边缘检测
def canny_demo(image):                                                                     # 定义一个名为canny_demo的函数
    t = 80                                                                                 # 设置Canny边缘检测的低阈值为80
    canny_output = cv.Canny(image, t, t * 2)                                               # 高阈值t * 2为160
    cv.imshow("canny_output", canny_output)                                                # 显示边缘检测结果
    #cv.imwrite("D:/canny_output.png", canny_output)                                       # 存储结果
    return canny_output                                                                    # 返回边缘检测后的二值图像（白色为边缘，黑色为背景）

src = cv.imread(PictureAddress)                                                            # 读取图像文件
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                                                # 创建可自动调整大小的窗口
cv.imshow("input", src)                                                                    # 显示图像
edge_binary = canny_demo(src)                                                              # 调用canny_demo函数
k = np.ones((3, 3), dtype=np.uint8)                                                        # 图像处理中用作卷积核
binary = cv.morphologyEx(edge_binary, cv.MORPH_DILATE, k)                                  # 膨胀操作

# 轮廓发现
contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)    # 从二值图像中提取所有外部轮廓
for c in range(len(contours)):                                                             # 遍历所有检测到的轮廓
    rect = cv.minAreaRect(contours[c])                                                     # cv.minAreaRect()用于获取轮廓最小外接旋转矩形的函数
    cx, cy = rect[0]                                                                       # 矩形中心点坐标 (x,y)
    ww, hh = rect[1]                                                                       # 矩形尺寸 (width, height)‌注意‌：ww和hh不区分长宽，需后续比较确定
    ratio = np.minimum(ww, hh) / np.maximum(ww, hh)                                        # 计算宽高比（Aspect Ratio）的归一化值，接近1.0：正方形/圆形，接近0.0：细长形
    print(ratio)
    mm = cv.moments(contours[c])                                                           # 计算轮廓的24个几何矩
    m00 = mm['m00']                                                                        # 提取零阶空间矩矩(m00)作为轮廓面积
    m10,m01 = mm['m10'], mm['m01']                                                         # 提取一阶矩(m10,m01)用于质心计算
    #计算轮廓质心坐标
    cx = np.int32(m10 / m00)                                                               # 质心X坐标（像素整型）
    cy = np.int32(m01 / m00)                                                               # 质心Y坐标
    box = cv.boxPoints(rect)                                                               # 获取矩形四个顶点
    box = np.int32(box)                                                                    # 转换为整数坐标
    if ratio > 0.9:                                                                        # 接近正方形的形状
        cv.drawContours(src, [box], 0, (0, 0, 255), 2)                                     # 红色矩形框
        cv.circle(src, (np.int32(cx), np.int32(cy)), 2, (255, 0, 0), 2, 8, 0)              # 蓝色质心点
    if ratio < 0.5:                                         # 细长形状
        cv.drawContours(src, [box], 0, (255, 0, 255), 2)                                   # 品红矩形框
        cv.circle(src, (np.int32(cx), np.int32(cy)), 2, (0, 0, 255), 2, 8, 0)              # 红色质心点
 
# 显示
cv.imshow("contours_analysis", src)
#cv.imwrite("D:/contours_analysis.png", src)

cv.waitKey(0)                                                                              # 等待按键
cv.destroyAllWindows()                                                                     # 关闭所有窗口